# Import libraries and setup¶

In [1]:
# Import libraries we may need
import scanpy as sc
import numpy as np
import scipy as sp
import squidpy as sq
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import scanpy.external as sce
import wget
import yaml
import wget
import astir
import dill
import umap
reducer = umap.UMAP()
import scvelo as scv


/Users/joaoluizsfilho/opt/anaconda3/envs/Squidpy/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/Users/joaoluizsfilho/opt/anaconda3/envs/Squidpy/lib/python3.9/site-packages/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()
/Users/joaoluizsfilho/opt/anaconda3/envs/Squidpy/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
OMP: Info #276: omp_set_nested routine deprecated, please u

In [2]:
import anndata as ad

In [3]:
# Set up output figure settings
plt.rcParams['figure.figsize']=(128,128) #rescale figures, increase sizehere

# Set up scanpy settings
sc.settings.verbosity = 3
sc.set_figure_params(dpi=100, dpi_save=300) #Increase DPI for better resolution figures
#sc.logging.print_versions()

# Save / restore environemnt

Use these functions to either save or load the environmental variables, otherwise you will lose all the objects between opening/closing Jupyter sessions!

In [ ]:
#save the session
dill.dump_session('BM_Lonza_dataset_scVelo.db')

In [ ]:
#load the session
dill.load_session('DC_SCANPY.db')

# Read the data

In [4]:
cd /Users/joaoluizsfilho/Library/CloudStorage/Dropbox/Work_Files/Matthias_Lab/Projects/Vivax_human_project/Results/Prospective_Study_1/BM_lonza/scRNAseq/Scanpy_analysis

/Users/joaoluizsfilho/Library/CloudStorage/Dropbox/Work_Files/Matthias_Lab/Projects/Vivax_human_project/Results/Prospective_Study_1/BM_lonza/scRNAseq/Scanpy_analysis


In [5]:
pwd

'/Users/joaoluizsfilho/Library/CloudStorage/Dropbox/Work_Files/Matthias_Lab/Projects/Vivax_human_project/Results/Prospective_Study_1/BM_lonza/scRNAseq/Scanpy_analysis'

In [ ]:
#Read your data file (loom, h5ad, csv, …) using:

adata = scv.read('./h5ad_files/Lonza_dataset/BM_query.h5ad', cache=True)

In [ ]:
adata

In [ ]:
adata.obs['sample_id']

# Load, combine and prepare adata for scVelo analysis - option 1

In [ ]:
# load and combine .loom files

files = ['./loom_files/HD_BM1.1.loom', './loom_files/HD_BM1.2.loom', './loom_files/HD_BM2.1.loom', './loom_files/HD_BM2.2.loom', './loom_files/HD_BM3.1.loom', './loom_files/HD_BM3.2.loom', './loom_files/HD_BM4.1.loom', './loom_files/HD_BM4.2.loom']

In [ ]:
import loompy
loompy.combine(files = files, output_file = './loom_files/Lonza_dataset.loom', key="Accession")

In [ ]:
#load combined loom file
Lonza_dataset = scv.read('./loom_files/Lonza_dataset.loom', cache=True)

In [ ]:
Lonza_dataset

In [ ]:
scv.pp.remove_duplicate_cells(Lonza_dataset)
Lonza_dataset.var_names_make_unique()

In [ ]:
Lonza_dataset

In [ ]:
scv.utils.clean_obs_names(adata)
scv.utils.clean_obs_names(Lonza_dataset)

In [ ]:
#see discussions on github: https://github.com/theislab/scvelo/issues/989
adata = scv.utils.merge(adata, Lonza_dataset)

In [ ]:
adata

In [ ]:
adata.obs

In [ ]:
Lonza_dataset.obs

In [ ]:
adata.write('./h5ad_files/Lonza_dataset/BM_query_scVelo.h5ad')

In [ ]:
adata2 = scv.read('./h5ad_files/Lonza_dataset/BM_query_scVelo.h5ad', cache=True)

In [ ]:
adata2

In [ ]:
adata2.obs['sample_id']

# Load, combine and prepare adata for scVelo analysis - option 2

In [ ]:
# load loom files for spliced/unspliced matrices for each sample:
ldata1 = scv.read('./loom_files/Sample_BM-1.loom', cache=True)
ldata2 = scv.read('./loom_files/Sample_BM-2.loom', cache=True)
ldata3 = scv.read('./loom_files/Sample_BM-3.loom', cache=True)
ldata4 = scv.read('./loom_files/Sample_BM-4.loom', cache=True)
ldata5 = scv.read('./loom_files/Sample_BM-5.loom', cache=True)
ldata6 = scv.read('./loom_files/Sample_BM-6.loom', cache=True)
ldata7 = scv.read('./loom_files/Sample_BM-7.loom', cache=True)
ldata8 = scv.read('./loom_files/Sample_BM-8.loom', cache=True)

In [ ]:
# rename barcodes in order to merge:
#barcodes = [bc.split(':')[1] for bc in ldata1.obs.index.tolist()]
#barcodes = [bc[0:len(bc)-1] + '_1' for bc in barcodes]
#ldata1.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata2.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_2' for bc in barcodes]
ldata2.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata3.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_3' for bc in barcodes]
ldata3.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata4.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_4' for bc in barcodes]
ldata4.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata5.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_5' for bc in barcodes]
ldata5.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata6.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_6' for bc in barcodes]
ldata6.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata7.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_7' for bc in barcodes]
ldata7.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata8.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_8' for bc in barcodes]
ldata8.obs.index = barcodes

In [ ]:
# make variable names unique
ldata1.var_names_make_unique()
ldata2.var_names_make_unique()
ldata3.var_names_make_unique()
ldata4.var_names_make_unique()
ldata5.var_names_make_unique()
ldata6.var_names_make_unique()
ldata7.var_names_make_unique()
ldata8.var_names_make_unique()

# concatenate the three loom
ldata = ldata1.concatenate([ldata2, ldata3, ldata4, ldata5, ldata6, ldata7, ldata8])


In [ ]:
ldata

In [ ]:
scv.pp.remove_duplicate_cells(ldata)

In [ ]:
scv.utils.clean_obs_names(ldata)

In [ ]:
adata2 = scv.utils.merge(adata, ldata)

In [ ]:
adata2

In [ ]:
adata.write('./h5ad_files/Lonza_dataset/BM_query_scVelo_v2.h5ad')

# Load, combine and prepare adata for scVelo analysis - option 1 (BM Lonza + JCI datasets)

In [6]:
#Read your data file (loom, h5ad, csv, …) using:

adata = scv.read('./h5ad_files/Merged_dataset/BM_merged_query_processed.h5ad', cache=True) # use the adata file after running DPT and PAGA

In [7]:
adata

AnnData object with n_obs × n_vars = 99956 × 25113
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'percent.mt', 'UMI', 'Genes', 'sample_id', 'experiment_date', 'percent_ribo', 'percent_hb', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.1', 'seurat_clusters', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.celltype.l2.score', 'predicted.celltype.l2', 'mapping.score', 'predicted.celltype.score', 'predicted.celltype', 'major_celltype.l2', 'predicted_ADT_snn_res.1', 'SCT.weight', 'predicted_ADT.weight', 'predicted.ADT.wsnn_res.1.5', 'SingleR.labels', 'SingleR.labels2', 'SingleR.labels3', 'harmony_clusters', 'WNN_clusters', 'CIPR_harmony_clusters', 'CIPR_WNN_clusters', 'dpt_pseudotime', 'palantir_pseudotime', 'palantir_entropy'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable', 'n_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'DM_EigenValues', 'SingleR.labels_colors'

In [ ]:
# load and combine .loom files

files = ['./loom_files/HD_BM1.1.loom', './loom_files/HD_BM1.2.loom', './loom_files/HD_BM2.1.loom', './loom_files/HD_BM2.2.loom', './loom_files/HD_BM3.1.loom', './loom_files/HD_BM3.2.loom', './loom_files/HD_BM4.1.loom', './loom_files/HD_BM4.2.loom', './loom_files/HD_BM9.loom', './loom_files/HD_BM10.loom', './loom_files/HD_BM11.loom', './loom_files/HD_BM12.loom', './loom_files/HD_BM13.loom', './loom_files/HD_BM14.loom', './loom_files/HD_BM15.loom', './loom_files/HD_BM16.loom', './loom_files/HD_BM17.loom', './loom_files/HD_BM18.loom', './loom_files/HD_BM19.loom', './loom_files/HD_BM20.loom', './loom_files/HD_BM21.loom', './loom_files/HD_BM22.loom', './loom_files/HD_BM23.loom', './loom_files/HD_BM24.loom', './loom_files/HD_BM25.loom', './loom_files/HD_BM26.loom', './loom_files/HD_BM27.loom', './loom_files/HD_BM28.loom', './loom_files/HD_BM29.loom', './loom_files/HD_BM30.loom', './loom_files/HD_BM31.loom', './loom_files/HD_BM32.loom', './loom_files/HD_BM33.loom']


In [ ]:
import loompy
loompy.combine(files = files, output_file = './loom_files/Merged_dataset.loom', key="Accession")

In [ ]:
#load combined loom file
Merged_dataset = scv.read('./loom_files/Merged_dataset.loom', cache=True)

In [ ]:
Merged_dataset

In [ ]:
scv.pp.remove_duplicate_cells(Merged_dataset)
Merged_dataset.var_names_make_unique()

In [ ]:
Merged_dataset

In [ ]:
scv.utils.clean_obs_names(adata)
scv.utils.clean_obs_names(Merged_dataset)

In [ ]:
adata

In [ ]:
Merged_dataset

In [ ]:
#see discussions on github: https://github.com/theislab/scvelo/issues/989
adata2 = scv.utils.merge(adata, Merged_dataset)

In [ ]:
adata2

In [ ]:
sc.pl.umap(adata, color="sample_id")

In [ ]:
adata.obs['orig.ident']

In [ ]:
adata.write('./h5ad_files/Merged_dataset/BM_merged_query_scVelo.h5ad')

In [ ]:
adata = scv.read('./h5ad_files/Merged_dataset/BM_merged_query_scVelo.h5ad', cache=True)

# Load, combine and prepare adata for scVelo analysis - option 2

In [59]:
# load loom files for spliced/unspliced matrices for each sample:
ldata1 = scv.read('./loom_files/HD_BM1.1.loom', cache=True)
ldata2 = scv.read('./loom_files/HD_BM1.2.loom', cache=True)
ldata3 = scv.read('./loom_files/HD_BM2.1.loom', cache=True)
ldata4 = scv.read('./loom_files/HD_BM2.2.loom', cache=True)
ldata5 = scv.read('./loom_files/HD_BM3.1.loom', cache=True)
ldata6 = scv.read('./loom_files/HD_BM3.2.loom', cache=True)
ldata7 = scv.read('./loom_files/HD_BM4.1.loom', cache=True)
ldata8 = scv.read('./loom_files/HD_BM4.2.loom', cache=True)
ldata9 = scv.read('./loom_files/HD_BM9.loom', cache=True)
ldata10 = scv.read('./loom_files/HD_BM10.loom', cache=True)
ldata11 = scv.read('./loom_files/HD_BM11.loom', cache=True)
ldata12 = scv.read('./loom_files/HD_BM12.loom', cache=True)
ldata13 = scv.read('./loom_files/HD_BM13.loom', cache=True)
ldata14 = scv.read('./loom_files/HD_BM14.loom', cache=True)
ldata15 = scv.read('./loom_files/HD_BM15.loom', cache=True)
ldata16 = scv.read('./loom_files/HD_BM16.loom', cache=True)
ldata17 = scv.read('./loom_files/HD_BM17.loom', cache=True)
ldata18 = scv.read('./loom_files/HD_BM18.loom', cache=True)
ldata19 = scv.read('./loom_files/HD_BM19.loom', cache=True)
ldata20= scv.read('./loom_files/HD_BM20.loom', cache=True)
ldata21 = scv.read('./loom_files/HD_BM21.loom', cache=True)
ldata22 = scv.read('./loom_files/HD_BM22.loom', cache=True)
ldata23 = scv.read('./loom_files/HD_BM23.loom', cache=True)
ldata24 = scv.read('./loom_files/HD_BM24.loom', cache=True)
ldata25 = scv.read('./loom_files/HD_BM25.loom', cache=True)
ldata26 = scv.read('./loom_files/HD_BM26.loom', cache=True)
ldata27 = scv.read('./loom_files/HD_BM27.loom', cache=True)
ldata28 = scv.read('./loom_files/HD_BM28.loom', cache=True)
ldata29 = scv.read('./loom_files/HD_BM29.loom', cache=True)
ldata30 = scv.read('./loom_files/HD_BM30.loom', cache=True)
ldata31 = scv.read('./loom_files/HD_BM31.loom', cache=True)
ldata32 = scv.read('./loom_files/HD_BM32.loom', cache=True)
ldata33 = scv.read('./loom_files/HD_BM33.loom', cache=True)


... reading from cache file cache/loom_files-HD_BM1.1.h5ad
... reading from cache file cache/loom_files-HD_BM1.2.h5ad
... reading from cache file cache/loom_files-HD_BM2.1.h5ad
... reading from cache file cache/loom_files-HD_BM2.2.h5ad
... reading from cache file cache/loom_files-HD_BM3.1.h5ad
... reading from cache file cache/loom_files-HD_BM3.2.h5ad
... reading from cache file cache/loom_files-HD_BM4.1.h5ad
... reading from cache file cache/loom_files-HD_BM4.2.h5ad
... reading from cache file cache/loom_files-HD_BM9.h5ad
... reading from cache file cache/loom_files-HD_BM10.h5ad
... reading from cache file cache/loom_files-HD_BM11.h5ad
... reading from cache file cache/loom_files-HD_BM12.h5ad
... reading from cache file cache/loom_files-HD_BM13.h5ad
... reading from cache file cache/loom_files-HD_BM14.h5ad
... reading from cache file cache/loom_files-HD_BM15.h5ad
... reading from cache file cache/loom_files-HD_BM16.h5ad
... reading from cache file cache/loom_files-HD_BM17.h5ad


KeyboardInterrupt: 

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata1.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_1' for bc in barcodes]
ldata1.obs.index = barcodes


In [63]:
ldata1.obs

""
CellID
Sample_BM-1:AACACACGTGGCTACCx
Sample_BM-1:AACAAAGGTACCGGAAx
Sample_BM-1:AACACACAGAAATGGGx
Sample_BM-1:AAAGGGCAGGTAAGAGx
Sample_BM-1:AACAAAGCACGTAGTTx
...
Sample_BM-1:TTTGGTTAGCTAGATAx
Sample_BM-1:TTTCATGTCTTCCTAAx
Sample_BM-1:TTTGATCGTAACAAGTx


In [60]:
barcodes = [bc.split(':')[1] for bc in ldata1.obs.index.tolist()]

In [61]:
barcodes

['AACACACGTGGCTACCx',
 'AACAAAGGTACCGGAAx',
 'AACACACAGAAATGGGx',
 'AAAGGGCAGGTAAGAGx',
 'AACAAAGCACGTAGTTx',
 'AACAACCGTAGGATATx',
 'AAAGTGACATCCGTGGx',
 'AAAGGATCACCATATGx',
 'AACAAGACACTACCGGx',
 'AACAGGGAGTCAGCGAx',
 'AACAAAGAGAGAGTGAx',
 'AAACGCTGTATACGGGx',
 'AAAGTGAGTCTTGAGTx',
 'AAACGCTGTCTCCTGTx',
 'AACACACCAGTGTGGAx',
 'AACAAGACAGGAATATx',
 'AACAAAGCAAATAAGCx',
 'AAAGTCCTCGTTCAGAx',
 'AAAGGGCCAGAGGTACx',
 'AAAGTGACAATTCACGx',
 'AAAGGTAAGCATGCGAx',
 'AACAAGAAGTTGCGAGx',
 'AAAGGGCGTCAACATCx',
 'AACAAAGAGTCAATCCx',
 'AAATGGACATCAGTGTx',
 'AAAGTGAAGCCTTCTCx',
 'AAAGAACCATTGTCGAx',
 'AAACCCATCGAACACTx',
 'AAACCCACAGTCTGGCx',
 'AAACGAAGTGGTCTGCx',
 'AACAAGATCAAAGGATx',
 'AAACGCTTCATGGTACx',
 'AAAGTGATCTTGGAACx',
 'AAACGCTGTGAGATCGx',
 'AAAGGATGTGACGCCTx',
 'AAAGTCCTCACTCCGTx',
 'AAAGGTAGTTATGTGCx',
 'AACAAGAGTAGACTGGx',
 'AAACGCTAGAAGCGAAx',
 'AACAGGGAGCGATGACx',
 'AAATGGACAGAATTCCx',
 'AAAGTGACATAATCCGx',
 'AACAACCAGGTACAATx',
 'AAACCCAAGTGATGGCx',
 'AACAACCCAAACTGCTx',
 'AAAGGTAC

In [64]:
barcodes = [bc[0:len(bc)-1] for bc in barcodes]

In [65]:
barcodes

['AACACACGTGGCTACC',
 'AACAAAGGTACCGGAA',
 'AACACACAGAAATGGG',
 'AAAGGGCAGGTAAGAG',
 'AACAAAGCACGTAGTT',
 'AACAACCGTAGGATAT',
 'AAAGTGACATCCGTGG',
 'AAAGGATCACCATATG',
 'AACAAGACACTACCGG',
 'AACAGGGAGTCAGCGA',
 'AACAAAGAGAGAGTGA',
 'AAACGCTGTATACGGG',
 'AAAGTGAGTCTTGAGT',
 'AAACGCTGTCTCCTGT',
 'AACACACCAGTGTGGA',
 'AACAAGACAGGAATAT',
 'AACAAAGCAAATAAGC',
 'AAAGTCCTCGTTCAGA',
 'AAAGGGCCAGAGGTAC',
 'AAAGTGACAATTCACG',
 'AAAGGTAAGCATGCGA',
 'AACAAGAAGTTGCGAG',
 'AAAGGGCGTCAACATC',
 'AACAAAGAGTCAATCC',
 'AAATGGACATCAGTGT',
 'AAAGTGAAGCCTTCTC',
 'AAAGAACCATTGTCGA',
 'AAACCCATCGAACACT',
 'AAACCCACAGTCTGGC',
 'AAACGAAGTGGTCTGC',
 'AACAAGATCAAAGGAT',
 'AAACGCTTCATGGTAC',
 'AAAGTGATCTTGGAAC',
 'AAACGCTGTGAGATCG',
 'AAAGGATGTGACGCCT',
 'AAAGTCCTCACTCCGT',
 'AAAGGTAGTTATGTGC',
 'AACAAGAGTAGACTGG',
 'AAACGCTAGAAGCGAA',
 'AACAGGGAGCGATGAC',
 'AAATGGACAGAATTCC',
 'AAAGTGACATAATCCG',
 'AACAACCAGGTACAAT',
 'AAACCCAAGTGATGGC',
 'AACAACCCAAACTGCT',
 'AAAGGTACACATTCTT',
 'AAACGCTGTCTCCCTA',
 'AAAGTCCGTAG

In [66]:
barcodes = ['HD_BM1.1_' + bc for bc in barcodes]

In [67]:
barcodes

['HD_BM1.1_AACACACGTGGCTACC',
 'HD_BM1.1_AACAAAGGTACCGGAA',
 'HD_BM1.1_AACACACAGAAATGGG',
 'HD_BM1.1_AAAGGGCAGGTAAGAG',
 'HD_BM1.1_AACAAAGCACGTAGTT',
 'HD_BM1.1_AACAACCGTAGGATAT',
 'HD_BM1.1_AAAGTGACATCCGTGG',
 'HD_BM1.1_AAAGGATCACCATATG',
 'HD_BM1.1_AACAAGACACTACCGG',
 'HD_BM1.1_AACAGGGAGTCAGCGA',
 'HD_BM1.1_AACAAAGAGAGAGTGA',
 'HD_BM1.1_AAACGCTGTATACGGG',
 'HD_BM1.1_AAAGTGAGTCTTGAGT',
 'HD_BM1.1_AAACGCTGTCTCCTGT',
 'HD_BM1.1_AACACACCAGTGTGGA',
 'HD_BM1.1_AACAAGACAGGAATAT',
 'HD_BM1.1_AACAAAGCAAATAAGC',
 'HD_BM1.1_AAAGTCCTCGTTCAGA',
 'HD_BM1.1_AAAGGGCCAGAGGTAC',
 'HD_BM1.1_AAAGTGACAATTCACG',
 'HD_BM1.1_AAAGGTAAGCATGCGA',
 'HD_BM1.1_AACAAGAAGTTGCGAG',
 'HD_BM1.1_AAAGGGCGTCAACATC',
 'HD_BM1.1_AACAAAGAGTCAATCC',
 'HD_BM1.1_AAATGGACATCAGTGT',
 'HD_BM1.1_AAAGTGAAGCCTTCTC',
 'HD_BM1.1_AAAGAACCATTGTCGA',
 'HD_BM1.1_AAACCCATCGAACACT',
 'HD_BM1.1_AAACCCACAGTCTGGC',
 'HD_BM1.1_AAACGAAGTGGTCTGC',
 'HD_BM1.1_AACAAGATCAAAGGAT',
 'HD_BM1.1_AAACGCTTCATGGTAC',
 'HD_BM1.1_AAAGTGATCTTGGAAC',
 'HD_BM1.1

In [68]:
ldata1.obs.index = barcodes

In [69]:
ldata1.obs

""
HD_BM1.1_AACACACGTGGCTACC
HD_BM1.1_AACAAAGGTACCGGAA
HD_BM1.1_AACACACAGAAATGGG
HD_BM1.1_AAAGGGCAGGTAAGAG
HD_BM1.1_AACAAAGCACGTAGTT
...
HD_BM1.1_TTTGGTTAGCTAGATA
HD_BM1.1_TTTCATGTCTTCCTAA
HD_BM1.1_TTTGATCGTAACAAGT
HD_BM1.1_TTTGTTGCAGCATTGT


In [25]:
# rename barcodes in order to merge:
barcodes = [bc.split(':')[1] for bc in ldata1.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_1' for bc in barcodes]
ldata1.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata2.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_2' for bc in barcodes]
ldata2.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata3.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_3' for bc in barcodes]
ldata3.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata4.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_4' for bc in barcodes]
ldata4.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata5.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_5' for bc in barcodes]
ldata5.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata6.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_6' for bc in barcodes]
ldata6.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata7.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_7' for bc in barcodes]
ldata7.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata8.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_8' for bc in barcodes]
ldata8.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata9.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_9' for bc in barcodes]
ldata9.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata10.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_10' for bc in barcodes]
ldata10.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata11.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_11' for bc in barcodes]
ldata11.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata12.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_12' for bc in barcodes]
ldata12.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata13.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_13' for bc in barcodes]
ldata13.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata14.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_14' for bc in barcodes]
ldata14.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata15.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_15' for bc in barcodes]
ldata15.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata16.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_16' for bc in barcodes]
ldata16.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata17.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_17' for bc in barcodes]
ldata17.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata18.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_18' for bc in barcodes]
ldata18.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata19.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_19' for bc in barcodes]
ldata19.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata20.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_20' for bc in barcodes]
ldata20.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata21.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_21' for bc in barcodes]
ldata21.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata22.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_22' for bc in barcodes]
ldata22.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata23.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_23' for bc in barcodes]
ldata23.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata24.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_24' for bc in barcodes]
ldata24.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata25.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_25' for bc in barcodes]
ldata25.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata26.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_26' for bc in barcodes]
ldata26.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata27.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_27' for bc in barcodes]
ldata27.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata28.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_28' for bc in barcodes]
ldata28.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata29.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_29' for bc in barcodes]
ldata29.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata30.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_30' for bc in barcodes]
ldata30.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata31.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_31' for bc in barcodes]
ldata31.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata32.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_32' for bc in barcodes]
ldata32.obs.index = barcodes

barcodes = [bc.split(':')[1] for bc in ldata33.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_33' for bc in barcodes]
ldata33.obs.index = barcodes

In [26]:
ldata33.obs

""
AAACCTGAGTAGATG_33
AAACCTGAGACGCAC_33
AAACCTGAGGATATA_33
AAACCTGAGGTGATT_33
AAACCTGCACGAGGT_33
...
TTTGTCAGTCTAGTG_33
TTTGTCAGTCCGTCA_33
TTTGTCAGTCAAACT_33
TTTGTCAGTTGCGCA_33


In [27]:
# make variable names unique
ldata1.var_names_make_unique()
ldata2.var_names_make_unique()
ldata3.var_names_make_unique()
ldata4.var_names_make_unique()
ldata5.var_names_make_unique()
ldata6.var_names_make_unique()
ldata7.var_names_make_unique()
ldata8.var_names_make_unique()
ldata9.var_names_make_unique()
ldata10.var_names_make_unique()
ldata11.var_names_make_unique()
ldata12.var_names_make_unique()
ldata13.var_names_make_unique()
ldata14.var_names_make_unique()
ldata15.var_names_make_unique()
ldata16.var_names_make_unique()
ldata17.var_names_make_unique()
ldata18.var_names_make_unique()
ldata19.var_names_make_unique()
ldata20.var_names_make_unique()
ldata21.var_names_make_unique()
ldata22.var_names_make_unique()
ldata23.var_names_make_unique()
ldata24.var_names_make_unique()
ldata25.var_names_make_unique()
ldata26.var_names_make_unique()
ldata27.var_names_make_unique()
ldata28.var_names_make_unique()
ldata29.var_names_make_unique()
ldata30.var_names_make_unique()
ldata31.var_names_make_unique()
ldata32.var_names_make_unique()
ldata33.var_names_make_unique()


In [28]:
ldata32

AnnData object with n_obs × n_vars = 9720 × 36601
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [29]:
ldata32.obs

""
AAACCTGAGGATATA_32
AAACCTGAGAGACTA_32
AAACCTGAGTGTACT_32
AAACCTGAGGCGACA_32
AAACCTGAGTGGTAG_32
...
TTTGTCAGTCAAACT_32
TTTGTCATCCTACAG_32
TTTGTCATCTTCATG_32
TTTGTCATCGTTGCC_32


In [70]:
scv.utils.clean_obs_names(ldata1)

In [73]:
ldata1.obs

,sample_batch
AACACACGTGGC,HD_BM1.1_TACC
AACAAAGGTACC,HD_BM1.1_GGAA
AACACACAGAAA,HD_BM1.1_TGGG
AAAGGGCAGGTA,HD_BM1.1_AGAG
AACAAAGCACGT,HD_BM1.1_AGTT
...,...
TTTGGTTAGCTA,HD_BM1.1_GATA
TTTCATGTCTTC,HD_BM1.1_CTAA
TTTGATCGTAAC,HD_BM1.1_AAGT
TTTGTTGCAGCA,HD_BM1.1_TTGT


In [30]:
# concatenate the three loom
ldata = ldata1.concatenate([ldata2, ldata3, ldata4, ldata5, ldata6, ldata7, ldata8, ldata9, ldata10, ldata11, ldata12, ldata13, ldata14, ldata15, ldata16, ldata17, ldata18, ldata19, ldata20, ldata21, ldata22, ldata23, ldata24, ldata25, ldata26, ldata27, ldata28, ldata29, ldata30, ldata31, ldata32, ldata33])

In [31]:
ldata.obs

,batch
AACACACGTGGCTACC_1-0,0
AACAAAGGTACCGGAA_1-0,0
AACACACAGAAATGGG_1-0,0
AAAGGGCAGGTAAGAG_1-0,0
AACAAAGCACGTAGTT_1-0,0
...,...
TTTGTCAGTCTAGTG_33-32,32
TTTGTCAGTCCGTCA_33-32,32
TTTGTCAGTCAAACT_33-32,32
TTTGTCAGTTGCGCA_33-32,32


In [ ]:
scv.pp.remove_duplicate_cells(ldata)

In [33]:
scv.utils.clean_obs_names(ldata)

In [34]:
ldata

AnnData object with n_obs × n_vars = 260637 × 36601
    obs: 'batch', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'sample_batch'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [35]:
ldata.obs

,batch,initial_size_unspliced,initial_size_spliced,initial_size,sample_batch
AACACACGTGGC,0,1792,1341,1341.0,TACC_1-0
AACAAAGGTACC,0,27,31,31.0,GGAA_1-0
AACACACAGAAA,0,2582,519,519.0,TGGG_1-0
AAAGGGCAGGTA,0,1880,3008,3008.0,AGAG_1-0
AACAAAGCACGT,0,10630,11982,11982.0,AGTT_1-0
...,...,...,...,...,...
TTTGTCAGTCTA-3,32,431,1315,1315.0,GTG_33-32
TTTGTCAGTCCG,32,287,808,808.0,TCA_33-32
TTTGTCAGTCAA-8,32,9,75,75.0,ACT_33-32
TTTGTCAGTTGC-1,32,1109,3115,3115.0,GCA_33-32


In [21]:
scv.utils.clean_obs_names(adata)

In [ ]:
adata

In [36]:
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,nCount_ADT,nFeature_ADT,percent.mt,UMI,Genes,sample_id,experiment_date,...,SingleR.labels2,SingleR.labels3,harmony_clusters,WNN_clusters,CIPR_harmony_clusters,CIPR_WNN_clusters,dpt_pseudotime,palantir_pseudotime,palantir_entropy,sample_batch
AAACCCAAGTGA,HD_BM1.1,4933.0,1962,640.0,10,3.547537,4933.0,1962,HD_BM1.1,09_22,...,B_cell:Naive,naive B-cells,5,23,Naive B cell,Naive B cell,0.369608,0.010191,1.212716,HD_BM1.1_TGGC-1
AAACCCACAGTC,HD_BM1.1,7980.0,2666,500.0,10,6.892231,7980.0,2666,HD_BM1.1,09_22,...,Pro-Myelocyte,GMP,25,47,CMP,CMP,0.481783,0.061754,1.249741,HD_BM1.1_TGGC-1
AAACCCAGTCTA,HD_BM1.1,6257.0,2244,633.0,10,2.669011,6257.0,2244,HD_BM1.1,09_22,...,B_cell:Naive,naive B-cells,5,23,Naive B cell,Naive B cell,0.463983,0.010075,1.212716,HD_BM1.1_ACTG-1
AAACCCAGTGCA,HD_BM1.1,13179.0,3697,675.0,10,1.380985,13179.0,3697,HD_BM1.1,09_22,...,NK_cell,Monocytes,29,59,Plasmacytoid DC,Plasmacytoid DC,0.439500,0.027786,1.213168,HD_BM1.1_TACT-1
AAACCCATCGAA,HD_BM1.1,5868.0,2062,639.0,10,5.299932,5868.0,2062,HD_BM1.1,09_22,...,T_cell:CD8+_effector_memory,CD8+ T-cells,10,14,CM CD8+ T cell,Naive CD4+ T cell,0.473968,0.005529,1.212694,HD_BM1.1_CACT-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCACACCCA,HD_BM32,4737.0,1151,0.0,0,3.124340,4737.0,1151,HD_BM32,12_18,...,B_cell:immature,Memory B-cells,5,1,Naive B cell,Naive B cell,0.348813,1.151317,0.815095,HD_BM32_TGTG-1
TTGTCACATCGA,HD_BM32,3270.0,1017,0.0,0,1.865443,3270.0,1017,HD_BM32,12_18,...,T_cell:CD4+_central_memory,CD4+ Tcm,0,10,CM CD8+ T cell,EM CD4+ T cell,0.366847,1.319166,0.695820,HD_BM32_TCGC-1
TTGTCACATTGA,HD_BM32,3768.0,1124,0.0,0,1.273885,3768.0,1124,HD_BM32,12_18,...,T_cell:CD4+,CD4+ Tem,9,10,Naive CD4+ T cell,EM CD4+ T cell,0.470843,1.326451,0.695477,HD_BM32_TGCT-1
TTGTCATCCTAC,HD_BM32,5268.0,1434,0.0,0,4.233106,5268.0,1434,HD_BM32,12_18,...,Monocyte:CD16-,Monocytes,7,5,Monocyte,Neutrophilic Metamyelocyte,0.771174,1.221793,0.092183,HD_BM32_TAGA-1


In [37]:
adata2 = scv.utils.merge(adata, ldata)

In [23]:
adata2

AnnData object with n_obs × n_vars = 24291 × 22128
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'percent.mt', 'UMI', 'Genes', 'sample_id', 'experiment_date', 'percent_ribo', 'percent_hb', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.1', 'seurat_clusters', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.celltype.l2.score', 'predicted.celltype.l2', 'mapping.score', 'predicted.celltype.score', 'predicted.celltype', 'major_celltype.l2', 'predicted_ADT_snn_res.1', 'SCT.weight', 'predicted_ADT.weight', 'predicted.ADT.wsnn_res.1.5', 'SingleR.labels', 'SingleR.labels2', 'SingleR.labels3', 'harmony_clusters', 'WNN_clusters', 'CIPR_harmony_clusters', 'CIPR_WNN_clusters', 'dpt_pseudotime', 'palantir_pseudotime', 'palantir_entropy', 'sample_batch', 'batch', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable', 'n_counts', 'highly_variable', 'm

In [ ]:
adata2.obs

# 1- Preprocessing

In [ ]:
# As a first step, we filter out genes that are not sufficiently expressed both unspliced and spliced RNA (here, at least 20). 
# Following, the cell size is normalized for both unspliced and spliced RNA, and counts in adata.X log1p transformed to reduce the effect of outliers. 
# Next, we also identify and filter for highly variable genes (here 5000).

scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes= 5000)

In [ ]:
adata

In [ ]:
# In the case of RNA velocity, we additionally smooth observations by the mean expression in their neighborhood. This can be done using scVelo’s moments function.
sc.tl.pca(adata)
sc.pp.neighbors(adata) # the results with harmony did not look good so I ran as indicated in the tutorial...
scv.pp.moments(adata, n_pcs=None, n_neighbors=None)

In [ ]:
adata

# 2- RNA velocity inference - EM model

In [ ]:
# In order to calculate RNA velocity with the EM model, the parameters of splicing kinetics need to be infered first. 
# The inference is taken care of by scVelo’s recover_dynamics function.
scv.tl.recover_dynamics(adata, n_jobs=8)

In [ ]:
adata

In [ ]:
# The parameters of the splicing model are inferred by maximizing a given likelihood. 
# To study which genes were fit most confidently by scVelo, we can study the corresponding phase portraits as well as the inferred trajectory (plotted in purple) and steady-state ratio (dashed purple line). 
top_genes = adata.var["fit_likelihood"].sort_values(ascending=False).index
scv.pl.scatter(adata, basis=top_genes[:5], frameon=False)

In [ ]:
# calculate both velocity and the projection onto our two-dimensional UMAP embedding.

scv.tl.velocity(adata, mode="dynamical")

In [ ]:
scv.tl.velocity_graph(adata, n_jobs=8)

In [ ]:
adata

In [ ]:
_, ax = plt.subplots(figsize=(8, 8))
scv.pl.velocity_embedding_stream(adata, basis="X_proj.umap", min_mass = 0, save = "RNAvelo_stream_proj.UMAP.pdf", ax=ax)
# color = 'SingleR.labels',

In [ ]:
_, ax = plt.subplots(figsize=(8, 8))
scv.pl.velocity_embedding_grid(adata, basis='X_proj.umap', arrow_size = 2, arrow_length = 2, save = "RNAvelo_stream_proj.UMAP_grid.pdf", ax=ax)

In [ ]:
scv.pl.velocity_embedding(adata, basis='X_proj.umap')

In [ ]:
adata

In [ ]:
scv.pl.paga(adata, basis='X_proj.umap', size=50, alpha=.1,
            min_edge_width=2, node_size_scale=1.5)

In [ ]:
scv.pl.velocity(adata, var_names=['gene_A', 'gene_B'], **params)
scv.pl.velocity_graph(adata, **params)

# 3- RNA velocity inference - Steady-state model - skip this!

In [ ]:
#scv.tl.velocity(adata, mode="deterministic")
#scv.tl.velocity_graph(adata, n_jobs=8)
#scv.pl.velocity_embedding_stream(adata, basis="X_proj.umap")